In [4]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
from sqlalchemy import create_engine
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from dash import dash_table
import numpy as np
from jupyter_dash import JupyterDash
import pdfkit
import base64
from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import pyodbc
from urllib.parse import quote_plus
import io
import oracledb
from datetime import datetime
from dash.exceptions import PreventUpdate

In [5]:
# Definición de la función de conexión
def create_oracle_connection1():
    try:
        oracledb.init_oracle_client(lib_dir=r"C:\Users\kings\Downloads\Nueva carpeta\instantclient_23_4")
        connection = oracledb.connect(
            user="User_oper",
            password="TmLQL$Yq.1",
            dsn="10.56.1.76:1527/WNET"
        )
        print("Conexión exitosa.")
        return connection
    except oracledb.DatabaseError as e:
        error, = e.args
        print(f"Error al conectar: {error.message}")
        return None

# Reemplazar fetch_data por una función de ejemplo
def fetch_data1():
    try:
        conn = create_oracle_connection1()
        if conn is None:
            raise ConnectionError("No se pudo establecer la conexión a la base de datos.")
        
                # Asegúrate de que las fechas estén en el formato correcto
        query = f"""

SELECT A.ATENAMBORICENASICOD AS ORIGEN,
       A.ATENAMBCENASICOD AS CENTRO,
       TO_CHAR(A.ATENAMBATENFEC, 'yyyymm') AS PERIODO,
       DECODE(Y.PERSEXOCOD, '0', '2', '1', '1') AS SEXO,
       CB.GRPETA1COD AS GETAREO,
       Y.PERSECNUM AS PACSECNUM,
       SR.SERVHOSCOD AS CODSER,
       SR.SERVHOSTIPPROCOD AS TIPPROF,
       'ATENME' AS ORIGEN_TIPO
  FROM CTAAM10 A
  LEFT OUTER JOIN CMAME10 X
    ON X.ORICENASICOD = A.ATENAMBORICENASICOD
   AND X.CENASICOD = A.ATENAMBCENASICOD
   AND X.ACTMEDNUM = A.ATENAMBNUM
  LEFT OUTER JOIN CMPER10 Y
    ON X.ACTMEDPACSECNUM = Y.PERSECNUM
  LEFT OUTER JOIN SGSS.CBGPE10 CB
    ON CB.GRPETAEDADCOD = X.ACTMEDEDADATEN
  LEFT OUTER JOIN CMSHO10 SR
    ON SR.SERVHOSCOD = X.ACTMEDSERVHOSCOD
 WHERE TRUNC(A.ATENAMBATENFEC) BETWEEN TO_DATE('01/08/2024', 'DD/MM/YYYY') AND
       TO_DATE('31/08/2024', 'DD/MM/YYYY')
   AND A.ATENAMBESTREGCOD = '1'
   AND Y.PERSECNUM IS NOT NULL
   
UNION ALL

--DETALLE_B1_ATENNOMEDICAS:
SELECT B.ATENOMORICENASICOD AS ORIGEN,
       B.ATENOMCENASICOD AS CENTRO,
       TO_CHAR(B.ATENOMFEC, 'yyyymm') AS PERIODO,
       DECODE(Y1.PERSEXOCOD, '0', '2', '1', '1') AS SEXO,
       CB1.GRPETA1COD AS GETAREO,
       Y1.PERSECNUM AS PACSECNUM,
       SR1.SERVHOSCOD AS CODSER,
       SR1.SERVHOSTIPPROCOD AS TIPPROF,
       'ATENOME' AS ORIGEN_TIPO
  FROM CTANM10 B
  LEFT OUTER JOIN CMAME10 X1
    ON X1.ORICENASICOD = B.ATENOMORICENASICOD
   AND X1.CENASICOD = B.ATENOMCENASICOD
   AND X1.ACTMEDNUM = B.ATENOMACTMEDNUM
  LEFT OUTER JOIN CMPER10 Y1
    ON X1.ACTMEDPACSECNUM = Y1.PERSECNUM
  LEFT OUTER JOIN SGSS.CBGPE10 CB1
    ON CB1.GRPETAEDADCOD = X1.ACTMEDEDADATEN
  LEFT OUTER JOIN CMSHO10 SR1
    ON SR1.SERVHOSCOD = X1.ACTMEDSERVHOSCOD
 WHERE B.ATENOMFEC BETWEEN TO_DATE('01/08/2024', 'DD/MM/YYYY') AND
       TO_DATE('31/08/2024', 'DD/MM/YYYY')
   AND B.ATENOMESTREGCOD = '1'
   AND Y1.PERSECNUM IS NOT NULL
  
UNION ALL

--DETALLE_B1_ATENODONTO:
SELECT CT.CITAMBORICENASICOD AS ORIGEN,
       CT.CITAMBCENASICOD AS CENTRO,
       TO_CHAR(CT.CITAMBPROCONFEC, 'YYYYMM') AS PERIODO,
       DECODE(Y2.PERSEXOCOD, '0', '2', '1', '1') AS SEXO,
       CB2.GRPETA1COD AS GETAREO,
       Y2.PERSECNUM AS PACSECNUM,
       SR2.SERVHOSCOD AS CODSER,
       SR2.SERVHOSTIPPROCOD AS TIPPROF,
       'ATEODON' AS ORIGEN_TIPO
  FROM CTCAM10 CT
  LEFT OUTER JOIN CMAME10 X2
    ON CT.CITAMBORICENASICOD = X2.ORICENASICOD
   AND CT.CITAMBCENASICOD = X2.CENASICOD
   AND CT.CITAMBNUM = X2.ACTMEDNUM
  LEFT OUTER JOIN CMPER10 Y2
    ON X2.ACTMEDPACSECNUM = Y2.PERSECNUM
  LEFT OUTER JOIN SGSS.CBGPE10 CB2
    ON CB2.GRPETAEDADCOD = X2.ACTMEDEDADATEN
  LEFT OUTER JOIN CMSHO10 SR2
    ON SR2.SERVHOSCOD = X2.ACTMEDSERVHOSCOD
 WHERE CT.CITAMBPROCONFEC BETWEEN TO_DATE('01/08/2024', 'DD/MM/YYYY') AND
       TO_DATE('31/08/2024', 'DD/MM/YYYY')
   AND CT.CITAMBSERVHOSCOD IN ('E11', 'E12', 'E19')
   AND CT.ESTCITCOD = '4'
   AND Y2.PERSECNUM IS NOT NULL
        """
        print("Consulta SQL generada:")
        print(query)  # Agrega esto para depurar la consulta SQL

        df = pd.read_sql(query, conn)
        print(f"Cantidad de filas obtenidas: {len(df)}")  # Agrega esto para ver cuántas filas se obtienen
    except Exception as e:
        print(f"Error: {e}")
        df = pd.DataFrame()
    finally:
        if conn:
            conn.close()  # Cerrar la conexión al final
    return df


# Reemplazar fetch_data por una función de ejemplo
def fetch_data2():
    try:
        conn = create_oracle_connection1()
        if conn is None:
            raise ConnectionError("No se pudo establecer la conexión a la base de datos.")
        
                # Asegúrate de que las fechas estén en el formato correcto
        query = f"""-- First Query
      SELECT A.ATENAMBORICENASICOD AS ORIGEN,
             A.ATENAMBCENASICOD AS CENTRO,
             TO_CHAR(A.ATENAMBATENFEC, 'yyyymm') AS PERIODO,
             DECODE(Y.PERSEXOCOD, '0', '2', '1', '1') AS SEXO,
             CB.GRPETA1COD AS GETAREO,
             Y.PERSECNUM AS PACSECNUM,
             DECODE(INSTR(DX.DIAGCOD, '.'), 0, DX.DIAGCOD || '.X', DX.DIAGCOD) AS CODDX,
             SR.SERVHOSTIPPROCOD AS TIPPROF,
             'ATENME' AS ORIGEN_TIPO
        FROM CTAAM10 A
        LEFT OUTER JOIN CMAME10 X
          ON X.ORICENASICOD = A.ATENAMBORICENASICOD
         AND X.CENASICOD = A.ATENAMBCENASICOD
         AND X.ACTMEDNUM = A.ATENAMBNUM
        LEFT OUTER JOIN CMPER10 Y
          ON X.ACTMEDPACSECNUM = Y.PERSECNUM
        LEFT OUTER JOIN SGSS.CBGPE10 CB
          ON CB.GRPETAEDADCOD = X.ACTMEDEDADATEN
        LEFT OUTER JOIN CMSHO10 SR
          ON SR.SERVHOSCOD = X.ACTMEDSERVHOSCOD
        LEFT OUTER JOIN CTDAA10 DX
          ON A.ATENAMBORICENASICOD = DX.ATENAMBORICENASICOD
         AND A.ATENAMBCENASICOD = DX.ATENAMBCENASICOD
         AND A.ATENAMBNUM = DX.ATENAMBNUM
       WHERE TRUNC(A.ATENAMBATENFEC) BETWEEN TO_DATE('01/08/2024', 'DD/MM/YYYY') AND
             TO_DATE('31/08/2024', 'DD/MM/YYYY')
         AND A.ATENAMBESTREGCOD = '1'
         AND Y.PERSECNUM IS NOT NULL
         AND '2' = DX.ATENAMBTIPODIAGCOD

      UNION ALL

      -- Second Query
      SELECT B.ATENOMORICENASICOD AS ORIGEN,
             B.ATENOMCENASICOD AS CENTRO,
             TO_CHAR(B.ATENOMFEC, 'yyyymm') AS PERIODO,
             DECODE(Y1.PERSEXOCOD, '0', '2', '1', '1') AS SEXO,
             CB1.GRPETA1COD AS GETAREO,
             Y1.PERSECNUM AS PACSECNUM,
             DECODE(INSTR(DX.ATENMDDIAGCOD, '.'), 0, DX.ATENMDDIAGCOD || '.X', DX.ATENMDDIAGCOD) AS CODDX,
             SR1.SERVHOSTIPPROCOD AS TIPPROF,
             'ATENOME' AS ORIGEN_TIPO
        FROM CTANM10 B
        LEFT OUTER JOIN CMAME10 X1
          ON X1.ORICENASICOD = B.ATENOMORICENASICOD
         AND X1.CENASICOD = B.ATENOMCENASICOD
         AND X1.ACTMEDNUM = B.ATENOMACTMEDNUM
        LEFT OUTER JOIN CMPER10 Y1
          ON X1.ACTMEDPACSECNUM = Y1.PERSECNUM
        LEFT OUTER JOIN SGSS.CBGPE10 CB1
          ON CB1.GRPETAEDADCOD = X1.ACTMEDEDADATEN
        LEFT OUTER JOIN CMSHO10 SR1
          ON SR1.SERVHOSCOD = X1.ACTMEDSERVHOSCOD
        LEFT OUTER JOIN CTDAN10 DX
          ON B.ATENOMORICENASICOD = DX.ATENOMORICENASICOD
         AND B.ATENOMCENASICOD = DX.ATENOMCENASICOD
         AND B.ATENOMACTMEDNUM = DX.ATENOMACTMEDNUM
       WHERE B.ATENOMFEC BETWEEN TO_DATE('01/08/2024', 'DD/MM/YYYY') AND
             TO_DATE('31/08/2024', 'DD/MM/YYYY')
         AND B.ATENOMESTREGCOD = '1'
         AND Y1.PERSECNUM IS NOT NULL
         AND '2' = DX.ATENMDTIPODIAGCOD
         
      UNION ALL
      
	      --DETALLE_B2_ATENODONTO:
	SELECT CT.CITAMBORICENASICOD AS ORIGEN,
	       CT.CITAMBCENASICOD AS CENTRO,
	       TO_CHAR(CT.CITAMBPROCONFEC, 'YYYYMM') AS PERIODO,
	       DECODE(Y1.PERSEXOCOD, '0', '2', '1', '1') AS SEXO,
	       CB1.GRPETA1COD AS GETAREO,
	       Y1.PERSECNUM AS PACSECNUM,
	       DECODE(INSTR(DX1.DIAGCOD, '.'), 0, DX1.DIAGCOD || '.X', DX1.DIAGCOD) AS CODDX,
	       SR1.SERVHOSTIPPROCOD AS TIPPROF,
	       'ATEODO' AS ORIGEN_TIPO
	  FROM CTCAM10 CT
	  LEFT OUTER JOIN CMAME10 X1
	    ON CT.CITAMBORICENASICOD = X1.ORICENASICOD
	   AND CT.CITAMBCENASICOD = X1.CENASICOD
	   AND CT.CITAMBNUM = X1.ACTMEDNUM
	  LEFT OUTER JOIN CMPER10 Y1
	    ON X1.ACTMEDPACSECNUM = Y1.PERSECNUM
	  LEFT OUTER JOIN SGSS.CBGPE10 CB1
	    ON CB1.GRPETAEDADCOD = X1.ACTMEDEDADATEN
	  LEFT OUTER JOIN CMSHO10 SR1
	    ON SR1.SERVHOSCOD = X1.ACTMEDSERVHOSCOD
	  LEFT OUTER JOIN CTDAO10 DX1
	    ON CT.CITAMBORICENASICOD = DX1.ATENODOORICENASICOD
	   AND CT.CITAMBCENASICOD = DX1.ATENODOCENASICOD
	   AND CT.CITAMBNUM = DX1.ATENODONUM
	 WHERE CT.CITAMBPROCONFEC BETWEEN TO_DATE('01/08/2024', 'DD/MM/YYYY') AND
	       TO_DATE('01/08/2024', 'DD/MM/YYYY')
	   AND CT.CITAMBSERVHOSCOD IN ('E11', 'E12', 'E19')
	   AND CT.ESTCITCOD = '4'
	   AND Y1.PERSECNUM IS NOT NULL
	   AND '2' = DX1.TIPODIAGCOD
        """

        print("Consulta SQL generada:")
        print(query)  # Agrega esto para depurar la consulta SQL

        df = pd.read_sql(query, conn)
        print(f"Cantidad de filas obtenidas: {len(df)}")  # Agrega esto para ver cuántas filas se obtienen
    except Exception as e:
        print(f"Error: {e}")
        df = pd.DataFrame()
    finally:
        if conn:
            conn.close()  # Cerrar la conexión al final
    return df

In [6]:
data=fetch_data1()
data2=fetch_data2()

Conexión exitosa.
Consulta SQL generada:


SELECT A.ATENAMBORICENASICOD AS ORIGEN,
       A.ATENAMBCENASICOD AS CENTRO,
       TO_CHAR(A.ATENAMBATENFEC, 'yyyymm') AS PERIODO,
       DECODE(Y.PERSEXOCOD, '0', '2', '1', '1') AS SEXO,
       CB.GRPETA1COD AS GETAREO,
       Y.PERSECNUM AS PACSECNUM,
       SR.SERVHOSCOD AS CODSER,
       SR.SERVHOSTIPPROCOD AS TIPPROF,
       'ATENME' AS ORIGEN_TIPO
  FROM CTAAM10 A
  LEFT OUTER JOIN CMAME10 X
    ON X.ORICENASICOD = A.ATENAMBORICENASICOD
   AND X.CENASICOD = A.ATENAMBCENASICOD
   AND X.ACTMEDNUM = A.ATENAMBNUM
  LEFT OUTER JOIN CMPER10 Y
    ON X.ACTMEDPACSECNUM = Y.PERSECNUM
  LEFT OUTER JOIN SGSS.CBGPE10 CB
    ON CB.GRPETAEDADCOD = X.ACTMEDEDADATEN
  LEFT OUTER JOIN CMSHO10 SR
    ON SR.SERVHOSCOD = X.ACTMEDSERVHOSCOD
 WHERE TRUNC(A.ATENAMBATENFEC) BETWEEN TO_DATE('01/08/2024', 'DD/MM/YYYY') AND
       TO_DATE('31/08/2024', 'DD/MM/YYYY')
   AND A.ATENAMBESTREGCOD = '1'
   AND Y.PERSECNUM IS NOT NULL
   
UNION ALL

--DETALLE_B1_ATENNOM

C:\Users\kings\AppData\Local\Temp\ipykernel_6728\3752005476.py:112: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Cantidad de filas obtenidas: 3760948
Conexión exitosa.
Consulta SQL generada:
-- First Query
      SELECT A.ATENAMBORICENASICOD AS ORIGEN,
             A.ATENAMBCENASICOD AS CENTRO,
             TO_CHAR(A.ATENAMBATENFEC, 'yyyymm') AS PERIODO,
             DECODE(Y.PERSEXOCOD, '0', '2', '1', '1') AS SEXO,
             CB.GRPETA1COD AS GETAREO,
             Y.PERSECNUM AS PACSECNUM,
             DECODE(INSTR(DX.DIAGCOD, '.'), 0, DX.DIAGCOD || '.X', DX.DIAGCOD) AS CODDX,
             SR.SERVHOSTIPPROCOD AS TIPPROF,
             'ATENME' AS ORIGEN_TIPO
        FROM CTAAM10 A
        LEFT OUTER JOIN CMAME10 X
          ON X.ORICENASICOD = A.ATENAMBORICENASICOD
         AND X.CENASICOD = A.ATENAMBCENASICOD
         AND X.ACTMEDNUM = A.ATENAMBNUM
        LEFT OUTER JOIN CMPER10 Y
          ON X.ACTMEDPACSECNUM = Y.PERSECNUM
        LEFT OUTER JOIN SGSS.CBGPE10 CB
          ON CB.GRPETAEDADCOD = X.ACTMEDEDADATEN
        LEFT OUTER JOIN CMSHO10 SR
          ON SR.SERVHOSCOD = X.ACTMEDSERVHOSCOD

C:\Users\kings\AppData\Local\Temp\ipykernel_6728\3752005476.py:233: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
